<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**노트북 2: 특화된 인코더 모델 탐색하기**</font>

이 노트북에서는 Hugging Face에서 제공하는 다양한 **사전학습 인코더 모델**을 불러와,  
텍스트가 어떻게 처리되는지, 각 모델이 어떤 입력과 구조를 가지는지 살펴봅니다.

#### **학습 목표:**

- Hugging Face에서 공개된 대표 인코더 모델(BERT 계열)을 탐색합니다.
- 모델 입력 흐름과 임베딩 구조를 분석하여 다양한 NLP 태스크에 대비합니다.

## **Part 2.1**: Hugging Face에서 사전학습 인코더 모델 가져오기

요즘은 모델을 처음부터 직접 만드는 대신, 이미 만들어진 오픈소스 모델들을 가져다 쉽게 활용할 수 있습니다.

Hugging Face는 다양한 Transformer 기반 사전학습 모델을 오픈소스로 제공합니다.  
우리는 이 중 대표적인 인코더 모델인 **BERT 계열**을 선택하여 활용할 것입니다.

<img src="imgs\huggingface_models.jpg" width="800">



#### 사용할 라이브러리: `transformers`

- Hugging Face에서 제공하는 대표 Python 라이브러리입니다.
- 이름 그대로, 우리가 배울 **Transformer 기반 모델들**(BERT, GPT 등)을 편리하게 사용할 수 있도록 도와줍니다.
- 특히 **pipeline API**는 복잡한 모델 내부 구조를 몰라도 입력 → 출력 흐름을 쉽게 체험할 수 있게 해줍니다.



#### 예시 모델: `bert-base-uncased`

- Hugging Face에서 가장 많이 사용되는 기본 BERT 모델입니다.
- `uncased`란 대소문자를 구분하지 않는다는 뜻입니다.

### **BERT란 무엇인가요?**

BERT는 "Bidirectional Encoder Representations from Transformers"의 줄임말로,  
**구글이 2018년에 공개한 대표적인 사전학습 언어모델(LLM)** 입니다.

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*Qww2aaIdqrWVeNmo3AS0ZQ.png">  

이미지 출처 : https://medium.com/data-science/a-complete-guide-to-bert-with-code-9f87602e4a11

#### 핵심 특징 요약

- **Transformer 인코더 구조만 사용**  
- **문맥을 양방향으로** 이해함 (앞·뒤 단어 모두 참고)
- 사전학습(pre-training) 후 다양한 NLP 작업에 쉽게 적용 가능

#### 사전학습 태스크

- **Masked Language Modeling (MLM)**: 문장 중 일부 단어를 [MASK]로 가리고, 그 자리에 어떤 단어가 적절한지 예측  
- **Next Sentence Prediction (NSP)**: 두 문장이 이어지는 문장인지 아닌지 예측


In [ ]:
# 1. Hugging Face의 transformers 라이브러리에서 pipeline을 import 합니다.
from transformers import pipeline


# pipeline을 통해 'fill-mask' 작업을 수행하는 모델을 초기화합니다
unmasker = pipeline(
    'fill-mask',  # fill-mask` 태스크는 문장 중 `[MASK]` 자리에 들어갈 단어를 예측하는 작업
    model='bert-base-uncased', 
    device='cuda',  # GPU 사용 설정 (작은 모델이라 CPU로도 충분히 실행 가능)
)

# 테스트 문장에서 [MASK] 토큰이 있는 부분을 문맥을 고려하여 가장 적절한 단어로 채웁니다.
unmasker("Hello I'm a [MASK] model.")

**결과 해석 :**
- score :	해당 단어가 들어갈 확률(신뢰도) (높을수록 더 자연스럽다고 판단한 단어)
- token :	해당 단어의 토큰 ID (BERT의 내부 사전 기준 숫자 ID)
- token_str :	예측된 단어(토큰을 사람이 읽을 수 있는 문자열로 디코딩한 결과)
- sequence :	[MASK] 자리에 이 단어를 넣은 완성된 문장

## **Part 2.2:** 모델 입력 처리 흐름 살펴보기

BERT와 같은 LLM은 텍스트를 숫자 시퀀스로 변환한 후 모델에 입력합니다.  
이 과정을 **토큰화(tokenization)** 라고 부르며, 모든 LLM의 기초입니다.

In [2]:
from transformers import pipeline
# BERT 모델을 사용하여 마스킹된 단어를 예측하는 파이프라인 생성
unmasker = pipeline('fill-mask', model='bert-base-uncased')  # 'fill-mask' 태스크는 문장에서 가려진(마스킹된) 단어를 예측하는 작업

# 토크나이저 테스트
msg = "Hello world!"

# 1. 토큰화: 입력 텍스트를 BERT가 이해할 수 있는 개별 토큰으로 분리
# 이를 통해 모델이 텍스트를 어떻게 나누어 처리하는지 확인할 수 있음
print("Tokenize:", unmasker.tokenizer.tokenize(msg))

# 2. 인코딩: 토큰을 숫자로 변환
# BERT 모델이 이해할 수 있는 숫자 형태로 변환하며, 시작/끝 등의 특수 토큰도 추가됨
encoded = unmasker.tokenizer.encode(msg)
print("Encoding:", encoded)

# 3. 디코딩: 숫자로 된 토큰을 다시 텍스트로 변환
# 인코딩된 숫자 시퀀스를 원래의 텍스트 형태로 복원
decoded = unmasker.tokenizer.decode(encoded)
print("Decoding:", decoded)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Tokenize: ['hello', 'world', '!']
Encoding: [101, 7592, 2088, 999, 102]
Decoding: [CLS] hello world! [SEP]


## **Part 2.3:** 모델 입력 구성 요소 확인

In [4]:
inputs = unmasker.tokenizer("Hello world!", "Have a great day!", return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102, 2031, 1037, 2307, 2154,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


| 구성 요소 | 설명 |
|-----------|------|
| `input_ids` | 단어(또는 subword)를 정수 ID로 바꾼 것 (모델 입력의 핵심) |
| `token_type_ids` | 문장 A와 문장 B를 구분하기 위한 플래그 (문장 A=0, B=1) |
| `attention_mask` | 마스크가 1인 부분만 모델이 주의(attend)함. 패딩 무시 등에 사용됨 |

→ 이 중 `input_ids`는 가장 핵심적인 정보입니다.

## **Part 2.4:** 인코더 모델의 임베딩 구조 분석

딥러닝 모델은 단순한 정수 ID(input_ids)를 그대로 사용하지 않습니다.  
→ 대신 각 토큰을 **고차원 벡터(임베딩)** 로 변환합니다.

####  왜 임베딩이 필요할까?

- 정수 ID는 순서만 의미할 뿐, 단어 간 의미 차이를 반영하지 못함  
- 임베딩은 **단어 간 의미적 유사성**을 수치 공간에 반영

####  예시

- "apple"과 "banana"는 비슷한 과일 → 임베딩 벡터 간 거리가 가까움  
- "apple"과 "car"는 관련 없음 → 임베딩 벡터 간 거리 멀어짐

In [5]:
model = unmasker.model  # masked language model의 기본 모델을 가져옵니다
model.bert.embeddings  # BERT의 임베딩 레이어에 접근합니다. 이 레이어는 입력 텍스트를 벡터로 변환하는 역할을 합니다

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

### BERT 모델의 임베딩 구성 요소
```text
최종 입력 임베딩 = 단어 임베딩 + 위치 임베딩 + 세그먼트 임베딩
```

- 단어 임베딩: 각 단어의 의미 표현
- 위치 임베딩: 단어의 순서 정보 (Transformer는 순서를 직접 알지 못하므로 필요)
- 세그먼트 임베딩: 문장 A인지 B인지 구분 (NSP 태스크에서 사용)

→ 이 3가지를 **element-wise 덧셈(add)** 으로 합쳐서  
Transformer 인코더의 첫 번째 레이어에 입력으로 넣습니다.

In [ ]:
# BERT의 임베딩 레이어는 3가지 정보를 더해서 최종 입력 벡터를 만듭니다

model.bert.embeddings.word_embeddings       # 단어 임베딩
model.bert.embeddings.position_embeddings   # 위치 임베딩
model.bert.embeddings.token_type_embeddings # 문장 유형 임베딩

## **Part 2.5:** 실제 문장의 임베딩 추출 예시

In [20]:
import torch  # PyTorch 딥러닝 프레임워크를 불러옵니다

# 임의의 예시 문장을 정의합니다
string = "Hello World From Me, my cat and my dog!"
# BERT 모델의 토크나이저를 변수에 할당합니다
tokenizer = unmasker.tokenizer

# 문장을 토큰화하고, 각 토큰을 사람이 읽을 수 있는 형태로 변환합니다
# convert_ids_to_tokens: 숫자 ID를 실제 단어 토큰으로 변환
# encode: 문장을 토큰 ID로 변환
tokens = [tokenizer.convert_ids_to_tokens(x) for x in tokenizer.encode(string)]

# 문장을 토큰 ID로 변환하고 PyTorch 텐서 형태로 변환합니다
# tokenizer()는 문장을 토큰화하고 여러 정보를 딕셔너리 형태로 반환하는데,
# 그 중 'input_ids' 키에 해당하는 토큰 ID들을 가져옵니다
token_ids = torch.tensor(tokenizer(string)['input_ids'])

# BERT 모델의 임베딩 레이어를 통해 각 토큰을 벡터로 변환합니다
# word_embeddings는 각 토큰을 고차원 벡터 공간에 매핑하는 역할을 합니다
embeddings = model.bert.embeddings.word_embeddings(token_ids)

# 임베딩된 결과의 형태(shape)를 출력합니다
# [문장 길이 x 임베딩 차원] 형태가 출력됩니다
print(embeddings.shape)

# 최종 임베딩 결과를 반환합니다
embeddings

torch.Size([13, 768])


tensor([[ 0.0136, -0.0265, -0.0235,  ...,  0.0087,  0.0071,  0.0151],
        [-0.0043, -0.0330, -0.0217,  ..., -0.0425, -0.0127, -0.0389],
        [ 0.0244,  0.0322,  0.0024,  ..., -0.0424, -0.0054, -0.0284],
        ...,
        [-0.0149,  0.0124,  0.0091,  ..., -0.0445, -0.0193,  0.0234],
        [ 0.0298, -0.0373, -0.0356,  ...,  0.0161,  0.0192,  0.0173],
        [-0.0145, -0.0100,  0.0060,  ..., -0.0250,  0.0046, -0.0015]],
       grad_fn=<EmbeddingBackward0>)

<div><img src="imgs/bert-construction.png" width="800"/></div>

# <font color="#76b900">**마무리**</font> 

우리는 bert-base-uncased 모델을 불러와 fill-mask 태스크로 예측을 수행해보았습니다.

입력 텍스트는 토큰화 후 input_ids, token_type_ids, attention_mask로 변환됩니다.

임베딩은 단어 + 위치 + 세그먼트 정보를 더한 벡터로 구성되며, 이후 Transformer 레이어로 전달됩니다.

다음 노트북에서는 인코더 모델로 시퀀스 분류, 토큰 분류, 범위 예측, 제로샷 분류 등 다양한 태스크를 수행해보겠습니다.

In [ ]:
## 실행 완료 후 커널을 종료해주세요!
import IPython  # IPython 인터프리터를 제어하기 위한 라이브러리를 임포트합니다

# 현재 실행 중인 IPython 커널의 인스턴스를 가져옵니다
app = IPython.Application.instance()

# 커널을 안전하게 종료합니다 (True는 재시작하지 않음을 의미)
app.kernel.do_shutdown(True)

<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>